In [16]:
import pandas as pd
data = pd.read_csv("flight_data_2021.csv")

In [17]:
columns_to_drop = ['Origin','Dest','DestState','OriginState',
                   'Operated_or_Branded_Code_Share_Partners',
                   'arr-type','arr-elevation_ft']
data = data.drop(columns_to_drop, axis=1)
data.head()

,Airline,Cancelled,Quarter,Month,DayOfWeek,DepDelayMinutes,DepTimeBlk,dept-type,dept-elevation_ft,Route_Popularity,Distance_Final,Wind_Speed_mph,Wind_Gust_mph,Visibility_miles,tempF,precip_in,daily_snow_in
0,SkyWest Airlines Inc.,False,1,3,3,0.0,0700-0759,medium_airport,2941.0,937.0,422.246813,10.0,17.0,6.0,49,0.0,0.0
1,SkyWest Airlines Inc.,False,1,3,3,0.0,0900-0959,large_airport,1135.0,937.0,422.246813,6.0,9.0,6.0,60,0.0,0.0
2,SkyWest Airlines Inc.,False,1,3,3,0.0,1300-1359,large_airport,266.0,918.0,1356.657710,10.0,18.0,6.0,39,0.0,0.0
3,SkyWest Airlines Inc.,False,1,3,3,0.0,1600-1659,large_airport,607.0,568.0,1396.572313,10.0,16.0,6.0,65,0.0,0.0
4,SkyWest Airlines Inc.,False,1,3,3,0.0,1800-1859,large_airport,1135.0,1135.0,684.252343,21.0,35.0,6.0,71,0.0,0.0


In [18]:
noncancelled_data = data[data['Cancelled']==False]

In [19]:
regression_df=noncancelled_data
# Drop the 'Cancelled' column
regression_df.drop('Cancelled', axis=1, inplace=True)

regression_df.head()

/tmp/ipykernel_168141/610716977.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_df.drop('Cancelled', axis=1, inplace=True)


,Airline,Quarter,Month,DayOfWeek,DepDelayMinutes,DepTimeBlk,dept-type,dept-elevation_ft,Route_Popularity,Distance_Final,Wind_Speed_mph,Wind_Gust_mph,Visibility_miles,tempF,precip_in,daily_snow_in
0,SkyWest Airlines Inc.,1,3,3,0.0,0700-0759,medium_airport,2941.0,937.0,422.246813,10.0,17.0,6.0,49,0.0,0.0
1,SkyWest Airlines Inc.,1,3,3,0.0,0900-0959,large_airport,1135.0,937.0,422.246813,6.0,9.0,6.0,60,0.0,0.0
2,SkyWest Airlines Inc.,1,3,3,0.0,1300-1359,large_airport,266.0,918.0,1356.657710,10.0,18.0,6.0,39,0.0,0.0
3,SkyWest Airlines Inc.,1,3,3,0.0,1600-1659,large_airport,607.0,568.0,1396.572313,10.0,16.0,6.0,65,0.0,0.0
4,SkyWest Airlines Inc.,1,3,3,0.0,1800-1859,large_airport,1135.0,1135.0,684.252343,21.0,35.0,6.0,71,0.0,0.0


In [20]:
regression_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5861080 entries, 0 to 5967955
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Airline            object 
 1   Quarter            int64  
 2   Month              int64  
 3   DayOfWeek          int64  
 4   DepDelayMinutes    float64
 5   DepTimeBlk         object 
 6   dept-type          object 
 7   dept-elevation_ft  float64
 8   Route_Popularity   float64
 9   Distance_Final     float64
 10  Wind_Speed_mph     float64
 11  Wind_Gust_mph      float64
 12  Visibility_miles   float64
 13  tempF              int64  
 14  precip_in          float64
 15  daily_snow_in      float64
dtypes: float64(9), int64(4), object(3)
memory usage: 760.2+ MB


In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Load your data
# regression_df = pd.read_csv('path_to_your_data.csv')

# Define categorical and numerical features
categorical_features = ['Airline', 'DepTimeBlk', 'dept-type']
numerical_features = ['Quarter', 'Month', 'DayOfWeek', 'dept-elevation_ft', 'Route_Popularity',
                      'Distance_Final', 'Wind_Speed_mph', 'Wind_Gust_mph', 'Visibility_miles', 
                      'tempF', 'precip_in', 'daily_snow_in']

# One-hot encode categorical variables
encoder = OneHotEncoder(sparse=False)
regression_df_categorical_encoded = pd.DataFrame(encoder.fit_transform(regression_df[categorical_features]), 
                                                 columns=encoder.get_feature_names_out(),
                                                 index=regression_df.index)  # Maintain the same index

# Concatenate encoded variables back to the DataFrame and drop original categorical columns
regression_df = pd.concat([regression_df.drop(categorical_features, axis=1), regression_df_categorical_encoded], axis=1)

# Normalize the dataset
scaler = MinMaxScaler()
regression_df[numerical_features] = scaler.fit_transform(regression_df[numerical_features])
scaled_target = np.log1p(regression_df['DepDelayMinutes'].values.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(regression_df[numerical_features + list(encoder.get_feature_names_out())], 
                                                    scaled_target, test_size=0.2, random_state=42)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Reshape data for LSTM [samples, time steps, features]
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

# Create DataLoader instances
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)

In [22]:
# This will give you the count of NaN values per column
nan_counts = regression_df.isna().sum()
print(nan_counts)

Quarter                                              0
Month                                                0
DayOfWeek                                            0
DepDelayMinutes                                      0
dept-elevation_ft                                    0
Route_Popularity                                     0
Distance_Final                                       0
Wind_Speed_mph                                       0
Wind_Gust_mph                                        0
Visibility_miles                                     0
tempF                                                0
precip_in                                            0
daily_snow_in                                        0
Airline_Air Wisconsin Airlines Corp                  0
Airline_Alaska Airlines Inc.                         0
Airline_Allegiant Air                                0
Airline_American Airlines Inc.                       0
Airline_Capital Cargo International                  0
Airline_Co

In [23]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2Model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [24]:
class TransformerForFlightDelay(nn.Module):
    def __init__(self, input_size, num_heads=5, num_layers=2, hidden_size=64, dropout=0.1):
        super(TransformerForFlightDelay, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=input_size,
                nhead=num_heads,
                dim_feedforward=hidden_size,
                dropout=dropout
            ),
            num_layers=num_layers
        )
        self.regressor = nn.Sequential(
            nn.LayerNorm(input_size),
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, input):
        output = self.transformer(input)
        pooled_output = output.mean(dim=1)  # Aggregate across time steps
        delay_prediction = self.regressor(pooled_output)
        return delay_prediction

In [25]:
# Initialize the model
model = TransformerForFlightDelay(input_size=X_train.size(-1))  # Input size is the number of features

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/nfs/home/ksk4532/.local/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [27]:
# Training loop
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    loss_history = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets.squeeze())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        loss_history.append(avg_loss)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

In [28]:
loss_history = train(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1, Loss: 2.051024915546758
Epoch 2, Loss: 2.0329523842165917
Epoch 3, Loss: 2.02871281627446
Epoch 4, Loss: 2.0266136513141455
Epoch 5, Loss: 2.0256491282234403
Epoch 6, Loss: 2.0250835195891552
Epoch 7, Loss: 2.024992040852111
Epoch 8, Loss: 2.02451046142946
Epoch 9, Loss: 2.0244489218807815
Epoch 10, Loss: 2.024843136762676


In [29]:
# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            predictions.extend(output.view(-1).tolist())  # Flatten and convert to list
            actuals.extend(target.view(-1).tolist())  # Flatten and convert to list
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    r2 = r2_score(actuals, predictions)
    return rmse, r2

In [30]:
evaluate_model(model, test_loader)

(1.4184644040247028, 0.13934720003027046)

In [97]:
class Attention(nn.Module):
    def __init__(self, input_size):
        super(Attention, self).__init__()
        self.input_size = input_size
        self.attention_weights = nn.Linear(self.input_size, 1)

    def forward(self, x):
        # Compute attention weights
        attn_scores = self.attention_weights(x)
        attn_weights = F.softmax(attn_scores, dim=1)
        return attn_weights

In [98]:
class SimAM_CNN_Attention_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SimAM_CNN_Attention_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.attention = Attention(64)  # Assuming hidden_size is the output size of LSTM
        self.lstm = nn.LSTM(input_size=64, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Convolutional layers
        x = x.permute(0, 2, 1)  # adjust dimensions for Conv1d
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.permute(0, 2, 1)  # revert dimensions for LSTM and attention

        # Apply attention mechanism before LSTM
        attention_weights = self.attention(x)  # [batch_size, seq_len, 1]

        # Apply attention to the input features
        attended_features = x * attention_weights  # Element-wise multiplication

        # LSTM layer
        lstm_out, _ = self.lstm(attended_features)  # [batch_size, seq_len, hidden_size]

        # Fully connected layer for classification
        x = self.fc(lstm_out[:, -1, :])  # Use the last output of the LSTM for classification
        return x

In [99]:
# Initialize model parameters
input_size = X_train.shape[2]  # Number of features per sequence step
hidden_size = 128  # Reduced hidden layer size for simplicity
num_layers = 1  # Reduce number of LSTM layers
num_classes = 1  # Output class for regression

model = SimAM_CNN_Attention_LSTM(input_size, hidden_size, num_layers, num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [100]:
def train_model(model, train_loader, optimizer, criterion, num_epochs=10, device='cpu'):
    model.train()
    for epoch in range(num_epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)  # Move data to GPU if available
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output.view(-1), target.view(-1))  # Flatten tensors
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}: Loss = {loss.item()}')

In [101]:
train_model(model, train_loader, optimizer, criterion, num_epochs=10)

Epoch 1: Loss = 2.5292818546295166
Epoch 2: Loss = 0.7702363133430481
Epoch 3: Loss = 1.6277503967285156
Epoch 4: Loss = 1.814877986907959
Epoch 5: Loss = 2.1251022815704346
Epoch 6: Loss = 0.9816793203353882
Epoch 7: Loss = 1.7906321287155151
Epoch 8: Loss = 1.7299578189849854
Epoch 9: Loss = 1.7723698616027832
Epoch 10: Loss = 2.681474208831787


In [102]:
# Evaluating the model
rmse, r_squared = evaluate_model(model, test_loader)
print(f'RMSE: {rmse:.4f}')
print(f'R-squared: {r_squared:.4f}')

RMSE: 1.4121
R-squared: 0.1470
